In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "iceberg_warehouse"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.6.1"

# Connect with Spark

In [2]:
config = {
    f"spark.sql.catalog.lakekeeper": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.lakekeeper.type": "rest",
    f"spark.sql.catalog.lakekeeper.uri": CATALOG_URL,
    f"spark.sql.catalog.lakekeeper.warehouse": WAREHOUSE,
    f"spark.sql.catalog.lakekeeper.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "lakekeeper",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}


In [3]:
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

spark.sql("USE lakekeeper")

DataFrame[]

## List Catalogs and Databases

In [4]:
spark.sql("SHOW CATALOGS").show(truncate=False)
print(f"Current default catalog: {spark.catalog.currentCatalog()}")
spark.sql("SHOW DATABASES").show(truncate=False)
spark.sql("SHOW TABLES in icebergdata").show(truncate=False)

+-------------+
|catalog      |
+-------------+
|lakekeeper   |
|spark_catalog|
+-------------+

Current default catalog: lakekeeper
+-----------+
|namespace  |
+-----------+
|icebergdata|
+-----------+

+-----------+------------------------------------------+-----------+
|namespace  |tableName                                 |isTemporary|
+-----------+------------------------------------------+-----------+
|icebergdata|debezium_offset_storage_table             |false      |
|icebergdata|debeziumcdc_dbz_inventory_products_on_hand|false      |
|icebergdata|debeziumcdc_dbz_inventory_orders          |false      |
|icebergdata|debeziumcdc_dbz_inventory_customers       |false      |
|icebergdata|debeziumcdc_dbz_inventory_geom            |false      |
|icebergdata|debeziumcdc_dbz_inventory_products        |false      |
+-----------+------------------------------------------+-----------+



## List Orders

In [9]:
# retrieve and print orders data
orders = spark.sql("select * from icebergdata.debeziumcdc_dbz_inventory_orders order by id desc")
orders.show(truncate=False)


+-----+----------+---------+--------+----------+---------+----+-------+-------------------+--------+
|id   |order_date|purchaser|quantity|product_id|__deleted|__op|__table|__source_ts_ns     |__db    |
+-----+----------+---------+--------+----------+---------+----+-------+-------------------+--------+
|10004|2016-02-21|1003     |1       |107       |false    |r   |orders |1764011890883201000|postgres|
|10003|2016-02-19|1002     |2       |106       |false    |r   |orders |1764011890883201000|postgres|
|10002|2016-01-17|1002     |2       |105       |false    |r   |orders |1764011890883201000|postgres|
|10001|2016-01-16|1001     |1       |102       |false    |r   |orders |1764011890883201000|postgres|
+-----+----------+---------+--------+----------+---------+----+-------+-------------------+--------+



## List Customers

In [11]:

# customers = spark.sql("select id,input_file_name() as input_file  from debeziumevents.debeziumcdc_testc_inventory_customers")
customers = spark.sql("select *  from icebergdata.debeziumcdc_dbz_inventory_customers order by 1 asc")
customers.limit(10).show(truncate=False)


+----+----------+---------+---------------------+---------+----+---------+-------------------+--------+
|id  |first_name|last_name|email                |__deleted|__op|__table  |__source_ts_ns     |__db    |
+----+----------+---------+---------------------+---------+----+---------+-------------------+--------+
|1001|Sally     |Thomas   |sally.thomas@acme.com|false    |r   |customers|1764011890883201000|postgres|
|1002|George    |Bailey   |gbailey@foobar.com   |false    |r   |customers|1764011890883201000|postgres|
|1003|Edward    |Walker   |ed@walker.com        |false    |r   |customers|1764011890883201000|postgres|
|1004|Anne      |Kretchmar|annek@noanswer.org   |false    |r   |customers|1764011890883201000|postgres|
+----+----------+---------+---------------------+---------+----+---------+-------------------+--------+



## Print Customer Table Schema

In [12]:
customers.printSchema()

root
 |-- id: integer (nullable = false)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- __deleted: string (nullable = true)
 |-- __op: string (nullable = true)
 |-- __table: string (nullable = true)
 |-- __source_ts_ns: long (nullable = true)
 |-- __db: string (nullable = true)



## Source Database Check Offset data

In [13]:
# retrieve and print offset
data = spark.sql("select * from icebergdata.debezium_offset_storage_table")
data.show(truncate=False)


+------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|id                                  |offset_data                                                                                                                                                                             |record_insert_ts          |
+------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|fc5fee91-26ef-4e65-af06-8002f58f96b8|{"[\"iceberg\",{\"server\":\"dbz\"}]":"{\"last_snapshot_record\":true,\"lsn\":33795320,\"txId\":758,\"ts_usec\":1764011890883201,\"snapshot\":\"INITIAL\",\"snapshot_completed\":true}"}|2025-11-24 19:18:15.8176

# Check Customers Table History

In [27]:
# retrieve and print offset
data = spark.sql(
    "select * from icebergdata.debeziumcdc_dbz_inventory_customers.history ORDER BY made_current_at DESC")
data.show(truncate=False)


+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-11-24 19:27:40.851|7180055507866554518|2130029796740343711|true               |
|2025-11-24 19:27:31.397|2130029796740343711|5486998555339858252|true               |
|2025-11-24 19:27:22.223|5486998555339858252|7795785864425554229|true               |
|2025-11-24 19:27:10.992|7795785864425554229|3824391931188094847|true               |
|2025-11-24 19:27:00.853|3824391931188094847|388697656969804389 |true               |
|2025-11-24 19:26:50.904|388697656969804389 |7748333104283076452|true               |
|2025-11-24 19:26:41.312|7748333104283076452|6396139167232603665|true               |
|2025-11-24 19:26:31.057|6396139167232603665|1199937389572754625|true               |
|2025-11-24 19:26:21.116|1199937389572754625|422028572

In [ ]:
# Check Customers Table Snapshots

In [29]:
# retrieve and print offset
data = spark.sql("select * from icebergdata.debeziumcdc_dbz_inventory_customers.snapshots")
data.limit(4).show(truncate=False)

+-----------------------+-------------------+-------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                                                         |summary                                                                                                                                                                 

In [ ]:
customers.printSchema()

In [ ]:
customers.printSchema()